<a href="https://colab.research.google.com/github/CarolBigData2016/google-colab/blob/master/01TF_CH4_4_intermediate_net_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intermediate Neural Network in TensorFlow

In this notebook, we convert our [intermediate-depth MNIST-classifying neural network](https://github.com/the-deep-learners/TensorFlow-LiveLessons/blob/master/notebooks/intermediate_net_in_keras.ipynb) from Keras to TensorFlow (compare them side by side) following Aymeric Damien's [Multi-Layer Perceptron Notebook](https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/3_NeuralNetworks/multilayer_perceptron.ipynb) style.

#### Load dependencies

In [1]:
import numpy as np
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

#### Load data

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
# no pre-processing necessary

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


#### Set neural network hyperparameters (tidier at top of file!)

In [3]:
lr = 0.1
epochs = 20
batch_size = 128
weight_initializer = tf.contrib.layers.xavier_initializer() # have to explicitly initialize xavier Glorot initialization, in keras as default.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



#### Set number of neurons for each layer

In [0]:
n_input = 784  # number of pixel for mnist dataset
n_dense_1 = 64 # Same as keras intermediate notebook
n_dense_2 = 64 # Same as keras intermediate notebook
n_classes = 10 # 10 classes of output

#### Define placeholders Tensors for inputs and labels

In [0]:
x = tf.placeholder(tf.float32, [None, n_input]) # make 2-dimensional for matrix multiplication
y = tf.placeholder(tf.float32, [None, n_classes])

#### Define types of layers

In [0]:
# dense layer with ReLU activation:
def dense(x, W, b):
    z = tf.add(tf.matmul(x, W), b)
    a = tf.nn.relu(z)
    return a

#### Design neural network architecture

In [0]:
def network(x, weights, biases):
    
    # two dense hidden layers: 
    dense_1 = dense(x, weights['W1'], biases['b1'])
    dense_2 = dense(dense_1, weights['W2'], biases['b2'])
    
    # linear output layer (softmax)
    out_layer_z = tf.add(tf.matmul(dense_2, weights['W_out']), biases['b_out'])
    
    return out_layer_z

#### Define dictionaries for storing weights and biases for each layer -- and initialize

In [0]:
bias_dict = {
    'b1': tf.Variable(tf.zeros([n_dense_1])), 
    'b2': tf.Variable(tf.zeros([n_dense_2])),
    'b_out': tf.Variable(tf.zeros([n_classes]))
}

weight_dict = {
    'W1': tf.get_variable('W1', [n_input, n_dense_1], initializer=weight_initializer),
    'W2': tf.get_variable('W2', [n_dense_1, n_dense_2], initializer=weight_initializer),
    'W_out': tf.get_variable('W_out', [n_dense_2, n_classes], initializer=weight_initializer)
}

#### Build model

In [0]:
predictions = network(x, weights=weight_dict, biases=bias_dict)

#### Define model's loss and its optimizer

In [10]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictions, labels=y)) #use cross_entropy on softmax layer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr).minimize(cost) # use gradient Descent Optimizer

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



#### Define evaluation metrics

In [0]:
# calculate accuracy by identifying test cases where the model's highest-probability class matches the true y label: 
correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(y, 1))
accuracy_pct = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) * 100 # What accuracy % did I get correct?

#### Create op for variable initialization

In [0]:
initializer_op = tf.global_variables_initializer()

#### Train the network in a session

In [13]:
with tf.Session() as session:
    session.run(initializer_op)
    
    print("Training for", epochs, "epochs.")
    
    # loop over epochs: 
    for epoch in range(epochs):
        
        avg_cost = 0.0 # track cost to monitor performance during training
        avg_accuracy_pct = 0.0
        
        # loop over all batches of an epoch:
        n_batches = int(mnist.train.num_examples / batch_size)
        for i in range(n_batches):
            
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            
            # feed batch data to run optimization and fetching cost and accuracy: 
            _, batch_cost, batch_acc = session.run([optimizer, cost, accuracy_pct], 
                                                   feed_dict={x: batch_x, y: batch_y})
            
            # accumulate mean loss and accuracy over epoch: 
            avg_cost += batch_cost / n_batches
            avg_accuracy_pct += batch_acc / n_batches
            
        # output logs at end of each epoch of training:
        # This needs to be done explicitly in Tensorflow, what done in Keras automatically
        print("Epoch ", '%03d' % (epoch+1), 
              ": cost = ", '{:.3f}'.format(avg_cost), 
              ", accuracy = ", '{:.2f}'.format(avg_accuracy_pct), "%", 
              sep='')
    
    print("Training Complete. Testing Model.\n")
    
    test_cost = cost.eval({x: mnist.test.images, y: mnist.test.labels})
    test_accuracy_pct = accuracy_pct.eval({x: mnist.test.images, y: mnist.test.labels})
    
    print("Test Cost:", '{:.3f}'.format(test_cost))
    print("Test Accuracy: ", '{:.2f}'.format(test_accuracy_pct), "%", sep='')

Training for 20 epochs.
Epoch 001: cost = 0.501, accuracy = 85.40%
Epoch 002: cost = 0.251, accuracy = 92.71%
Epoch 003: cost = 0.194, accuracy = 94.35%
Epoch 004: cost = 0.159, accuracy = 95.38%
Epoch 005: cost = 0.134, accuracy = 96.06%
Epoch 006: cost = 0.117, accuracy = 96.59%
Epoch 007: cost = 0.104, accuracy = 96.96%
Epoch 008: cost = 0.093, accuracy = 97.21%
Epoch 009: cost = 0.084, accuracy = 97.52%
Epoch 010: cost = 0.077, accuracy = 97.73%
Epoch 011: cost = 0.070, accuracy = 97.92%
Epoch 012: cost = 0.064, accuracy = 98.09%
Epoch 013: cost = 0.059, accuracy = 98.25%
Epoch 014: cost = 0.055, accuracy = 98.36%
Epoch 015: cost = 0.051, accuracy = 98.52%
Epoch 016: cost = 0.047, accuracy = 98.61%
Epoch 017: cost = 0.044, accuracy = 98.72%
Epoch 018: cost = 0.040, accuracy = 98.89%
Epoch 019: cost = 0.037, accuracy = 98.99%
Epoch 020: cost = 0.035, accuracy = 99.03%
Training Complete. Testing Model.

Test Cost: 0.084
Test Accuracy: 97.30%
